Implementing and Managing a Cached Function for Optimised Performance

In [1]:
from functools import lru_cache

@lru_cache(maxsize=128)
def cached_function(args):
    # This function is decorated with @lru_cache, which enables caching of its results.
    # The 'maxsize' parameter specifies that up to 128 results can be stored in the cache.
    # Caching can significantly improve performance by avoiding redundant calculations for repeated inputs.
    # When the function is called with the same arguments, the cached result is returned instead of recomputing it.
    # The function implementation should be inserted here.
    pass

# The cache associated with the 'cached_function' is cleared.
# This is useful in scenarios where the cached results are no longer valid or when it is necessary to free up memory.
cached_function.cache_clear()


Implementation of agggregated approach for forecasting asylum applications using ML models

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import random
from sklearn.preprocessing import RobustScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import explained_variance_score, mean_squared_error, mean_absolute_error, median_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit
import xgboost as xgb
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LSTM, Attention, Input, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
import matplotlib

# Setting the font to Times New Roman globally
matplotlib.rcParams['font.family'] = 'Times New Roman'

# Set random seeds for reproducibility
random_seed = 42
np.random.seed(random_seed)
tf.random.set_seed(random_seed)
random.seed(random_seed)

# Ensure reproducibility with TensorFlow 2.x
tf.keras.utils.set_random_seed(random_seed)
tf.config.experimental.enable_op_determinism()

# Create output directory for graphs
output_dir = "Aggregated Approach Outputs"
os.makedirs(output_dir, exist_ok=True)

# Load and preprocess data
data = pd.read_csv("final_thesis_data.csv")
data['year_month'] = pd.to_datetime(data['year_month'])
data = data.sort_values(by=['country', 'year_month'])

# Aggregate data by country and year_month
data_agg = data.groupby(['country', 'year_month']).agg({
    'asy_applications': 'sum',
    'illegal_border_crossings': 'mean',
    'push_factor_index': 'mean',
    'push_factor_index_high_level': 'mean',
    'deaths_civilians': 'mean',
    'gdp_per_capita_current_usd': 'mean',
    'gdp_per_capita_growth': 'mean',
    'regime_end_type': 'mean',
    'state_fiscal_source_revenue': 'mean',
    'state_authority_over_territory': 'mean',
    'political_polarisation': 'mean',
    'political_violence': 'mean',
    'domestic_autonomy': 'mean',
    'rule_of_law': 'mean',
}).reset_index()

# Define variables to lag and create lagged variables
variables_to_lag = {
    "illegal_border_crossings": [3],
    "push_factor_index": [3],
    "push_factor_index_high_level": [2],
    "deaths_civilians": [12],
    "gdp_per_capita_current_usd": [12],
    "gdp_per_capita_growth": [12],
    "regime_end_type": [0],
    "state_fiscal_source_revenue": [0],
    "state_authority_over_territory": [0],
    "political_polarisation": [0],
    "political_violence": [0],
    "domestic_autonomy": [0],
    "rule_of_law": [0],
}

# Adjust lagged variables at the country level
for var, lags in variables_to_lag.items():
    for lag in lags:
        lagged_var_name = f"{var}_lag_{lag}"
        data_agg[lagged_var_name] = data_agg.groupby(['country'])[var].shift(lag)

# Define rolling window sizes
rolling_windows = [3, 6, 12, 24]

# Calculate rolling mean and standard deviation for asylum applications
for window in rolling_windows:
    data_agg[f'asy_applications_rolling_mean_{window}'] = data_agg.groupby(['country'])['asy_applications'].transform(lambda x: x.rolling(window=window, min_periods=1).mean())
    data_agg[f'asy_applications_rolling_sd_{window}'] = data_agg.groupby(['country'])['asy_applications'].transform(lambda x: x.rolling(window=window, min_periods=1).std())

# Calculate rolling mean and standard deviation for push_factor_index
for window in rolling_windows:
    data_agg[f'push_factor_index_rolling_mean_{window}'] = data_agg.groupby(['country'])['push_factor_index'].transform(lambda x: x.rolling(window=window, min_periods=1).mean())
    data_agg[f'push_factor_index_rolling_sd_{window}'] = data_agg.groupby(['country'])['push_factor_index'].transform(lambda x: x.rolling(window=window, min_periods=1).std())

# Calculate exponential moving average for asylum applications
data_agg['asy_applications_ewm_mean'] = data_agg.groupby(['country'])['asy_applications'].transform(lambda x: x.ewm(span=12, adjust=False).mean())
data_agg['asy_applications_ewm_std'] = data_agg.groupby(['country'])['asy_applications'].transform(lambda x: x.ewm(span=12, adjust=False).std())

# Add temporal features
data_agg['month'] = data_agg['year_month'].dt.month
data_agg['month_sin'] = np.sin(2 * np.pi * data_agg['month'] / 12)
data_agg['month_cos'] = np.cos(2 * np.pi * data_agg['month'] / 12)

# Define the predictors and the target variable
predictors = [
    "illegal_border_crossings_lag_3", "push_factor_index_lag_3",
    "push_factor_index_high_level_lag_2", "deaths_civilians_lag_12",
    "gdp_per_capita_current_usd_lag_12", "gdp_per_capita_growth_lag_12",
    "regime_end_type_lag_0", "state_fiscal_source_revenue_lag_0",
    "state_authority_over_territory_lag_0", "political_polarisation_lag_0",
    "political_violence_lag_0", "domestic_autonomy_lag_0",
    "rule_of_law_lag_0",
    "asy_applications_rolling_mean_3", "asy_applications_rolling_sd_3",
    "asy_applications_rolling_mean_6", "asy_applications_rolling_sd_6",
    "asy_applications_rolling_mean_12", "asy_applications_rolling_sd_12",
    "asy_applications_rolling_mean_24", "asy_applications_rolling_sd_24",
    "push_factor_index_rolling_mean_3", "push_factor_index_rolling_sd_3",
    "push_factor_index_rolling_mean_6", "push_factor_index_rolling_sd_6",
    "push_factor_index_rolling_mean_12", "push_factor_index_rolling_sd_12",
    "push_factor_index_rolling_mean_24", "push_factor_index_rolling_sd_24",
    "asy_applications_ewm_mean", "asy_applications_ewm_std",
    "month_sin", "month_cos"
]
target_var = "asy_applications"

# Handle missing values using Iterative Imputer
imputer = IterativeImputer(max_iter=10, random_state=random_seed)
data_agg[predictors] = imputer.fit_transform(data_agg[predictors])

# Scale the predictors and target variable
scaler_x = RobustScaler()
scaler_y = RobustScaler()
data_agg[predictors] = scaler_x.fit_transform(data_agg[predictors])
data_agg[target_var] = scaler_y.fit_transform(data_agg[target_var].values.reshape(-1, 1)).flatten()

# Function to create and train the LSTM model with L2 regularization, increased dropout rate, and attention mechanism
def create_and_train_lstm_with_attention(X_train, y_train, X_val, y_val, units_1=32, units_2=32, dropout_rate=0.5, learning_rate=0.001, l2_lambda=0.01):
    inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))
    
    lstm_out = LSTM(units_1, return_sequences=True, kernel_regularizer=l2(l2_lambda))(inputs)
    lstm_out = Dropout(dropout_rate)(lstm_out)
    
    lstm_out = LSTM(units_2, return_sequences=True, kernel_regularizer=l2(l2_lambda))(lstm_out)
    lstm_out = Dropout(dropout_rate)(lstm_out)
    
    attention_out = Attention()([lstm_out, lstm_out])
    attention_out = Dense(units_2, activation='relu')(attention_out)
    attention_out = Flatten()(attention_out)
    
    outputs = Dense(1, kernel_regularizer=l2(l2_lambda), activation='linear')(attention_out)  # Use linear activation
    
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')
    
    early_stopping_lstm = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stopping_lstm], verbose=0)
    
    return model, history

# Function to create and train a neural network model with best hyperparameters, L2 regularization, and increased dropout rate
def create_best_nn_model(X_train, y_train, X_val, y_val, dropout_rate=0.3, units_1=128, units_2=64, learning_rate=0.001, l2_lambda=0.01):
    model = Sequential([
        Input(shape=(X_train.shape[1],)),  # Ensure this is the first layer
        Dense(units_1, activation='relu', kernel_regularizer=l2(l2_lambda)),
        BatchNormalization(),
        Dropout(dropout_rate),
        Dense(units_2, activation='relu', kernel_regularizer=l2(l2_lambda)),
        BatchNormalization(),
        Dropout(dropout_rate),
        Dense(1, kernel_regularizer=l2(l2_lambda), activation='linear')  # Use linear activation
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')
    early_stopping_nn = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    history = model.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stopping_nn], verbose=0)
    
    return model, history

# Function to train an XGBoost model with the best hyperparameters
def train_xgboost_model(X_train, y_train, X_val, y_val):
    model = xgb.XGBRegressor(
        learning_rate=0.2,
        max_depth=7,
        n_estimators=100,
        objective='reg:squarederror',
        random_state=random_seed,
        early_stopping_rounds=10
    )
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    
    return model 

# Function to split data for training and validation
def split_train_val(data, quarter='Q1'):
    data['quarter'] = data['year_month'].dt.to_period('Q')
    val_data = data[data['quarter'].astype(str).str.endswith(quarter)]
    train_data = data[~data.index.isin(val_data.index)]
    return train_data, val_data

# Function to calculate confidence intervals
def calculate_confidence_intervals(predictions, confidence_level=0.95):
    ### Calculate the confidence intervals for the predictions.###
    mean_preds = np.mean(predictions, axis=0)
    std_dev = np.std(predictions, axis=0)
    z_score = 1.96  # Corresponds to 95% confidence interval
    lower_bound = mean_preds - z_score * std_dev
    upper_bound = mean_preds + z_score * std_dev
    return mean_preds, lower_bound, upper_bound

# Function for bootstrapping predictions
def bootstrap_predictions(model_func, X_train, y_train, X_val, y_val, X_test, n_iterations=20):
    #### Generate bootstrapped predictions for a given model function.###
    predictions = []

    for _ in range(n_iterations):
        # Resample with replacement
        train_indices = np.random.choice(range(len(X_train)), size=len(X_train), replace=True)
        X_train_resample = X_train[train_indices]
        y_train_resample = y_train[train_indices]

        # Reshape for LSTM if needed
        if model_func.__name__ == 'create_and_train_lstm_with_attention':
            X_train_resample = np.reshape(X_train_resample, (X_train_resample.shape[0], 1, X_train_resample.shape[1]))
            X_val_resample = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))
            X_test_resample = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
        else:
            X_val_resample = X_val  # No reshaping needed for non-LSTM models
            X_test_resample = X_test

        # Train model on resampled data
        if model_func.__name__ in ['create_and_train_lstm_with_attention', 'create_best_nn_model']:
            model, _ = model_func(X_train_resample, y_train_resample, X_val_resample, y_val)
        else:
            model = model_func(X_train_resample, y_train_resample, X_val_resample, y_val)

        # Predict on test data
        preds = model.predict(X_test_resample).flatten()
        predictions.append(preds)

    return np.array(predictions)

# Function to compute weights based on RMSE
def calculate_weights(rmse_values):
    ## Calculate weights inversely proportional to the RMSE.###
    inv_rmse = 1 / np.array(rmse_values)
    return inv_rmse / np.sum(inv_rmse)

# Function to plot combined residuals over time
def plot_combined_residuals(all_residuals):
    countries = all_residuals['country'].unique()
    for country in countries:
        plt.figure(figsize=(15, 10))
        group_data = all_residuals[all_residuals['country'] == country]

        # Plot residuals for each dataset with different markers and colors
        train_data = group_data[group_data['data_split'] == 'train']
        val_data = group_data[group_data['data_split'] == 'val']
        test_data = group_data[group_data['data_split'] == 'test']

        plt.scatter(train_data['year_month'], train_data['Residuals'], alpha=0.5, label='Train', marker='o', color='blue')
        plt.scatter(val_data['year_month'], val_data['Residuals'], alpha=0.5, label='Validation', marker='x', color='green')
        plt.scatter(test_data['year_month'], test_data['Residuals'], alpha=0.5, label='Test', marker='^', color='red')

        plt.title(f'Residuals - {country}')
        plt.xlabel('Date')
        plt.ylabel('Residuals')
        plt.axhline(0, color='red', linestyle='--')
        plt.legend()
        plt.grid(True)

        plt.tight_layout()
        plt.savefig(f'{output_dir}/Residuals_{country}.png')
        plt.close()

# Function to plot average feature importances for each month
def plot_aggregated_feature_importances(xgboost_models, predictors, output_dir, month):
    # Initialise a list to hold the aggregated importances
    aggregated_importances = np.zeros(len(predictors))
    
    # Compute the mean importances for this month's models
    importances = np.array([model.feature_importances_ for model in xgboost_models])
    mean_importances = np.mean(importances, axis=0)
    
    # Sorting indices for plotting (highest to lowest)
    sorted_indices = np.argsort(mean_importances)[::-1]
    
    # Creating the plot
    plt.figure(figsize=(10, 8))
    plt.barh(np.array(predictors)[sorted_indices], mean_importances[sorted_indices], color='skyblue')
    plt.xlabel('Average Feature Importance')
    plt.ylabel('Features')
    plt.title(f'Average Feature Importance for {month}')
    plt.gca().invert_yaxis()  # Show most important feature at the top
    plt.tight_layout()
    
    # Save the plot to specified directory
    plt.savefig(f'{output_dir}/Aggregated_Feature_Importances_{month}.png')
    plt.close()

# Initialise DataFrames to store explained variance and metrics
explained_variance_df = pd.DataFrame(columns=['Date', 'Explained Variance XGBoost', 'Explained Variance LSTM', 'Explained Variance NN', 'Explained Variance Ensemble'])
metrics_per_country_list = []

# Define prediction dates
prediction_dates = ['2024-01-01', '2024-02-01', '2024-03-01']  

# TimeSeriesSplit object
tscv = TimeSeriesSplit(n_splits=5)

# Loop through each prediction date and generate forecasts
for date in prediction_dates:
    print(f"Processing predictions for {date}")
    
    # Split the data into training and testing sets
    train_data, val_data = split_train_val(data_agg, quarter='Q4')
    test_data = data_agg[data_agg['year_month'] == date]
    
    X_train = train_data[predictors]
    y_train = train_data[target_var]
    X_val = val_data[predictors]
    y_val = val_data[target_var]
    X_test = test_data[predictors]
    y_test = test_data[target_var]
    
    # Perform walk-forward validation
    lstm_history = []
    nn_history = []
    xgboost_models = []

    # Convert to NumPy arrays
    X_train_np = X_train.to_numpy()
    y_train_np = y_train.to_numpy()
    X_val_np = X_val.to_numpy()
    y_val_np = y_val.to_numpy()
    X_test_np = X_test.to_numpy()
    y_test_np = y_test.to_numpy()

    # Initialise lists to store RMSE values
    lstm_rmse_train = []
    nn_rmse_train = []
    xgboost_rmse_train = []

    lstm_rmse_val = []
    nn_rmse_val = []
    xgboost_rmse_val = []

    for train_index, val_index in tscv.split(X_train_np):
        X_train_cv, X_val_cv = X_train_np[train_index], X_train_np[val_index]
        y_train_cv, y_val_cv = y_train_np[train_index], y_train_np[val_index]
        
        X_train_lstm = np.reshape(X_train_cv, (X_train_cv.shape[0], 1, X_train_cv.shape[1]))
        X_val_lstm = np.reshape(X_val_cv, (X_val_cv.shape[0], 1, X_val_cv.shape[1]))
        
        num_models = 5
        
        lstm_models = []
        for i in range(num_models):
            # Set a new random seed for each model based on the loop index
            seed = random_seed + i
            np.random.seed(seed)
            tf.random.set_seed(seed)
            random.seed(seed)
            tf.keras.utils.set_random_seed(seed)

            model, history = create_and_train_lstm_with_attention(X_train_lstm, y_train_cv, X_val_lstm, y_val_cv)
            lstm_models.append(model)
            lstm_history.append(history)

            # Compute RMSE for train and validation sets
            train_pred_lstm = model.predict(X_train_lstm).flatten()
            val_pred_lstm = model.predict(X_val_lstm).flatten()

            lstm_rmse_train.append(np.sqrt(mean_squared_error(y_train_cv, train_pred_lstm)))
            lstm_rmse_val.append(np.sqrt(mean_squared_error(y_val_cv, val_pred_lstm)))
        
        nn_models = []
        for i in range(num_models):
            seed = random_seed + i
            np.random.seed(seed)
            tf.random.set_seed(seed)
            random.seed(seed)
            tf.keras.utils.set_random_seed(seed)
            
            model, history = create_best_nn_model(X_train_cv, y_train_cv, X_val_cv, y_val_cv)
            nn_models.append(model)
            nn_history.append(history)

            # Compute RMSE for train and validation sets
            train_pred_nn = model.predict(X_train_cv).flatten()
            val_pred_nn = model.predict(X_val_cv).flatten()

            nn_rmse_train.append(np.sqrt(mean_squared_error(y_train_cv, train_pred_nn)))
            nn_rmse_val.append(np.sqrt(mean_squared_error(y_val_cv, val_pred_nn)))

        xgboost_models_this_month = []
        for i in range(num_models):
            seed = random_seed + i
            np.random.seed(seed)
            tf.random.set_seed(seed)
            random.seed(seed)
            tf.keras.utils.set_random_seed(seed)
            
            model = train_xgboost_model(X_train_cv, y_train_cv, X_val_cv, y_val_cv)
            xgboost_models_this_month.append(model)

            # Compute RMSE for train and validation sets
            train_pred_xgb = model.predict(X_train_cv)
            val_pred_xgb = model.predict(X_val_cv)

            xgboost_rmse_train.append(np.sqrt(mean_squared_error(y_train_cv, train_pred_xgb)))
            xgboost_rmse_val.append(np.sqrt(mean_squared_error(y_val_cv, val_pred_xgb)))

        xgboost_models.append(xgboost_models_this_month)

    # Calculate average RMSE over all models
    avg_lstm_rmse_train = np.mean(lstm_rmse_train)
    avg_nn_rmse_train = np.mean(nn_rmse_train)
    avg_xgboost_rmse_train = np.mean(xgboost_rmse_train)

    avg_lstm_rmse_val = np.mean(lstm_rmse_val)
    avg_nn_rmse_val = np.mean(nn_rmse_val)
    avg_xgboost_rmse_val = np.mean(xgboost_rmse_val)

    # Calculate weights based on the average RMSE
    train_weights = calculate_weights([avg_xgboost_rmse_train, avg_lstm_rmse_train, avg_nn_rmse_train])
    val_weights = calculate_weights([avg_xgboost_rmse_val, avg_lstm_rmse_val, avg_nn_rmse_val])

    # Generate bootstrapped predictions for each model
    lstm_bootstrap_preds = bootstrap_predictions(
        create_and_train_lstm_with_attention, X_train_np, y_train_np, X_val_np, y_val_np, X_test_np
    )
    nn_bootstrap_preds = bootstrap_predictions(
        create_best_nn_model, X_train_np, y_train_np, X_val_np, y_val_np, X_test_np
    )
    xgboost_bootstrap_preds = bootstrap_predictions(
        train_xgboost_model, X_train_np, y_train_np, X_val_np, y_val_np, X_test_np
    )

    # Calculate CI widths and choose the model with the widest CI for ensemble predictions
    lstm_mean, lstm_lower_ci, lstm_upper_ci = calculate_confidence_intervals(lstm_bootstrap_preds)
    nn_mean, nn_lower_ci, nn_upper_ci = calculate_confidence_intervals(nn_bootstrap_preds)
    xgboost_mean, xgboost_lower_ci, xgboost_upper_ci = calculate_confidence_intervals(xgboost_bootstrap_preds)

    lstm_ci_width = lstm_upper_ci - lstm_lower_ci
    nn_ci_width = nn_upper_ci - nn_lower_ci
    xgboost_ci_width = xgboost_upper_ci - xgboost_lower_ci

    # Create an array to hold ensemble predictions using the model with the widest CI
    ensemble_mean_widest_ci = np.zeros_like(lstm_mean)
    ensemble_lower_ci = np.zeros_like(lstm_lower_ci)  # Define these arrays to store ensemble CI data
    ensemble_upper_ci = np.zeros_like(lstm_upper_ci)

    # Selecting the model with the widest CI for each instance
    for i in range(len(ensemble_mean_widest_ci)):
        if lstm_ci_width[i] >= nn_ci_width[i] and lstm_ci_width[i] >= xgboost_ci_width[i]:
            ensemble_mean_widest_ci[i] = lstm_mean[i]
            ensemble_lower_ci[i] = lstm_lower_ci[i]
            ensemble_upper_ci[i] = lstm_upper_ci[i]
        elif nn_ci_width[i] >= lstm_ci_width[i] and nn_ci_width[i] >= xgboost_ci_width[i]:
            ensemble_mean_widest_ci[i] = nn_mean[i]
            ensemble_lower_ci[i] = nn_lower_ci[i]
            ensemble_upper_ci[i] = nn_upper_ci[i]
        else:
            ensemble_mean_widest_ci[i] = xgboost_mean[i]
            ensemble_lower_ci[i] = xgboost_lower_ci[i]
            ensemble_upper_ci[i] = xgboost_upper_ci[i]

    # Rescale predictions
    y_test_rescaled = scaler_y.inverse_transform(y_test_np.reshape(-1, 1)).flatten()
    ensemble_pred_rescaled = np.round(scaler_y.inverse_transform(ensemble_mean_widest_ci.reshape(-1, 1)).flatten())

    results_df = test_data[['country', 'year_month']].copy()
    results_df['True Values'] = np.round(y_test_rescaled[:results_df.shape[0]])
    results_df['Ensemble Prediction'] = np.round(ensemble_pred_rescaled[:results_df.shape[0]])

    # Add lower and upper CIs for each model and the ensemble
    results_df['XGBoost Lower CI'] = np.round(scaler_y.inverse_transform(xgboost_lower_ci.reshape(-1, 1)).flatten()[:results_df.shape[0]])
    results_df['XGBoost Upper CI'] = np.round(scaler_y.inverse_transform(xgboost_upper_ci.reshape(-1, 1)).flatten()[:results_df.shape[0]])

    results_df['LSTM Lower CI'] = np.round(scaler_y.inverse_transform(lstm_lower_ci.reshape(-1, 1)).flatten()[:results_df.shape[0]])
    results_df['LSTM Upper CI'] = np.round(scaler_y.inverse_transform(lstm_upper_ci.reshape(-1, 1)).flatten()[:results_df.shape[0]])

    results_df['NN Lower CI'] = np.round(scaler_y.inverse_transform(nn_lower_ci.reshape(-1, 1)).flatten()[:results_df.shape[0]])
    results_df['NN Upper CI'] = np.round(scaler_y.inverse_transform(nn_upper_ci.reshape(-1, 1)).flatten()[:results_df.shape[0]])
    
    results_df['Ensemble Lower CI'] = np.round(scaler_y.inverse_transform(ensemble_lower_ci.reshape(-1, 1)).flatten()[:results_df.shape[0]])
    results_df['Ensemble Upper CI'] = np.round(scaler_y.inverse_transform(ensemble_upper_ci.reshape(-1, 1)).flatten()[:results_df.shape[0]])

    results_df['Residuals'] = results_df['True Values'] - results_df['Ensemble Prediction']

    # Store individual model predictions
    results_df['XGBoost Prediction'] = np.round(scaler_y.inverse_transform(xgboost_mean.reshape(-1, 1)).flatten()[:results_df.shape[0]])
    results_df['LSTM Prediction'] = np.round(scaler_y.inverse_transform(lstm_mean.reshape(-1, 1)).flatten()[:results_df.shape[0]])
    results_df['NN Prediction'] = np.round(scaler_y.inverse_transform(nn_mean.reshape(-1, 1)).flatten()[:results_df.shape[0]])
    
    # Calculate residuals for training and validation sets
    train_pred_xgb_list = [model.predict(X_train_np) for model in xgboost_models_this_month]
    train_pred_xgb = np.mean(train_pred_xgb_list, axis=0)
    
    train_pred_lstm_list = [model.predict(np.reshape(X_train_np, (X_train_np.shape[0], 1, X_train_np.shape[1]))).flatten() for model in lstm_models]
    train_pred_lstm = np.mean(train_pred_lstm_list, axis=0)
    
    train_pred_nn_list = [model.predict(X_train_np).flatten() for model in nn_models]
    train_pred_nn = np.mean(train_pred_nn_list, axis=0)

    # Compute weighted average for train predictions
    ensemble_train_pred = (train_weights[0] * train_pred_xgb +
                           train_weights[1] * train_pred_lstm +
                           train_weights[2] * train_pred_nn)

    # Rescale train predictions
    train_pred_rescaled = np.round(scaler_y.inverse_transform(ensemble_train_pred.reshape(-1, 1)).flatten())
    y_train_rescaled = scaler_y.inverse_transform(y_train_np.reshape(-1, 1)).flatten()
    train_residuals = y_train_rescaled - train_pred_rescaled
    train_data['Residuals'] = train_residuals

    # Calculate residuals for validation set
    val_pred_xgb_list = [model.predict(X_val_np) for model in xgboost_models_this_month]
    val_pred_xgb = np.mean(val_pred_xgb_list, axis=0)

    val_pred_lstm_list = [model.predict(np.reshape(X_val_np, (X_val_np.shape[0], 1, X_val_np.shape[1]))).flatten() for model in lstm_models]
    val_pred_lstm = np.mean(val_pred_lstm_list, axis=0)
    
    val_pred_nn_list = [model.predict(X_val_np).flatten() for model in nn_models]
    val_pred_nn = np.mean(val_pred_nn_list, axis=0)

    # Compute weighted average for validation predictions
    ensemble_val_pred = (val_weights[0] * val_pred_xgb +
                         val_weights[1] * val_pred_lstm +
                         val_weights[2] * val_pred_nn)

    val_pred_rescaled = np.round(scaler_y.inverse_transform(ensemble_val_pred.reshape(-1, 1)).flatten())
    y_val_rescaled = scaler_y.inverse_transform(y_val_np.reshape(-1, 1)).flatten()
    val_residuals = y_val_rescaled - val_pred_rescaled
    val_data['Residuals'] = val_residuals

    # Append residuals to the all_residuals DataFrame
    test_residuals = results_df['True Values'] - results_df['Ensemble Prediction']
    test_data['Residuals'] = test_residuals
    
    # Add a column to identify data splits
    train_data['data_split'] = 'train'
    val_data['data_split'] = 'val'
    test_data['data_split'] = 'test'

    # Initialise DataFrame to store all residuals
    all_residuals = pd.DataFrame()

    # Concatenate train, validation, and test data
    all_residuals = pd.concat([
        all_residuals,
        train_data[['country', 'year_month', 'Residuals', 'data_split']],
        val_data[['country', 'year_month', 'Residuals', 'data_split']],
        test_data[['country', 'year_month', 'Residuals', 'data_split']]
    ])

    groups = results_df.groupby(['country'])
    for country, group_data in groups:
        print(f"Residuals for Country: {country}")
        print(group_data[['year_month', 'Residuals']])

    # Calculate metrics for each country
    metrics_per_country = []
    for country, group_data in groups:
        y_true_country = group_data['True Values'].values
        y_pred_country = group_data['Ensemble Prediction'].values
        y_pred_xgb_country = group_data['XGBoost Prediction'].values
        y_pred_lstm_country = group_data['LSTM Prediction'].values
        y_pred_nn_country = group_data['NN Prediction'].values
        
        if not np.any(np.isnan(y_true_country)) and not np.any(np.isnan(y_pred_country)):
            mse_country = mean_squared_error(y_true_country, y_pred_country)
            rmse_country = np.sqrt(mse_country)  
            mae_country = mean_absolute_error(y_true_country, y_pred_country)
            mdae_country = median_absolute_error(y_true_country, y_pred_country)
            mape_country = mean_absolute_percentage_error(y_true_country + 1e-6, y_pred_country)

            mse_xgb_country = mean_squared_error(y_true_country, y_pred_xgb_country)
            rmse_xgb_country = np.sqrt(mse_xgb_country)
            mae_xgb_country = mean_absolute_error(y_true_country, y_pred_xgb_country)
            mdae_xgb_country = median_absolute_error(y_true_country, y_pred_xgb_country)
            mape_xgb_country = mean_absolute_percentage_error(y_true_country + 1e-6, y_pred_xgb_country)

            mse_lstm_country = mean_squared_error(y_true_country, y_pred_lstm_country)
            rmse_lstm_country = np.sqrt(mse_lstm_country)
            mae_lstm_country = mean_absolute_error(y_true_country, y_pred_lstm_country)
            mdae_lstm_country = median_absolute_error(y_true_country, y_pred_lstm_country)
            mape_lstm_country = mean_absolute_percentage_error(y_true_country + 1e-6, y_pred_lstm_country)

            mse_nn_country = mean_squared_error(y_true_country, y_pred_nn_country)
            rmse_nn_country = np.sqrt(mse_nn_country)
            mae_nn_country = mean_absolute_error(y_true_country, y_pred_nn_country)
            mdae_nn_country = median_absolute_error(y_true_country, y_pred_nn_country)
            mape_nn_country = mean_absolute_percentage_error(y_true_country + 1e-6, y_pred_nn_country)
            
            metrics_per_country.append({
                'Country': country,
                'MSE Ensemble': np.round(mse_country),
                'RMSE Ensemble': np.round(rmse_country),
                'MAE Ensemble': np.round(mae_country),
                'MDAE Ensemble': np.round(mdae_country),
                'MAPE Ensemble': np.round(mape_country),
                'MSE XGBoost': np.round(mse_xgb_country),
                'RMSE XGBoost': np.round(rmse_xgb_country),
                'MAE XGBoost': np.round(mae_xgb_country),
                'MDAE XGBoost': np.round(mdae_xgb_country),
                'MAPE XGBoost': np.round(mape_xgb_country),
                'MSE LSTM': np.round(mse_lstm_country),
                'RMSE LSTM': np.round(rmse_lstm_country),
                'MAE LSTM': np.round(mae_lstm_country),
                'MDAE LSTM': np.round(mdae_lstm_country),
                'MAPE LSTM': np.round(mape_lstm_country),
                'MSE NN': np.round(mse_nn_country),
                'RMSE NN': np.round(rmse_nn_country),
                'MAE NN': np.round(mae_nn_country),
                'MDAE NN': np.round(mdae_nn_country),
                'MAPE NN': np.round(mape_nn_country)
            })

    metrics_country_df = pd.DataFrame(metrics_per_country)
    metrics_per_country_list.append(metrics_country_df)
    
    # Save country-level metrics to CSV
    metrics_country_df.to_csv(f'{output_dir}/performance_metrics_country_{date}.csv', index=False)

    # Aggregate to total by summing up the country-level metrics
    overall_metrics = metrics_country_df.sum(numeric_only=True).to_dict()
    overall_metrics_df = pd.DataFrame([overall_metrics])
    overall_metrics_df.to_csv(f'{output_dir}/performance_metrics_overall_{date}.csv', index=False)

    # Plot feature importances for each month
    plot_aggregated_feature_importances(xgboost_models_this_month, predictors, output_dir, date)

    # Calculate explained variance for XGBoost, LSTM, NN, and Ensemble
    explained_variance_xgb = explained_variance_score(y_test_rescaled, results_df['XGBoost Prediction'].values)
    explained_variance_lstm = explained_variance_score(y_test_rescaled, results_df['LSTM Prediction'].values)
    explained_variance_nn = explained_variance_score(y_test_rescaled, results_df['NN Prediction'].values)
    explained_variance_ensemble = explained_variance_score(y_test_rescaled, ensemble_pred_rescaled)

    # Add explained variance to DataFrame
    explained_variance_df = pd.concat([explained_variance_df, pd.DataFrame({
        'Date': [date],
        'Explained Variance XGBoost': [explained_variance_xgb],
        'Explained Variance LSTM': [explained_variance_lstm],
        'Explained Variance NN': [explained_variance_nn],
        'Explained Variance Ensemble': [explained_variance_ensemble]
    })], ignore_index=True)

    print(f"Explained Variance - XGBoost: {explained_variance_xgb:.4f}")
    print(f"Explained Variance - LSTM: {explained_variance_lstm:.4f}")
    print(f"Explained Variance - NN: {explained_variance_nn:.4f}")
    print(f"Explained Variance - Ensemble: {explained_variance_ensemble:.4f}")

    # Aggregate true values and predicted values at the country and month level
    country_month_aggregation = results_df.groupby(['country', 'year_month']).sum(numeric_only=True).reset_index()
    country_month_aggregation.to_csv(f'{output_dir}/country_level_aggregation_{date}.csv', index=False)

    # Aggregate true values and predicted values overall by summing up the country-month-level metrics
    overall_aggregation = country_month_aggregation.sum(numeric_only=True).to_dict()
    overall_aggregation_df = pd.DataFrame([overall_aggregation])
    overall_aggregation_df.to_csv(f'{output_dir}/overall_aggregation_{date}.csv', index=False)

# Save explained variance for each model per month
explained_variance_df.to_csv(f'{output_dir}/explained_variance_by_model_per_month.csv', index=False)

# Plot combined residuals over time for each group after all forecasts
plot_combined_residuals(all_residuals)

# Plot learning curves for LSTM models
plt.figure(figsize=(14, 8))
for i, history in enumerate(lstm_history):
    plt.plot(history.history['loss'], label=f'LSTM Train {i+1}', linestyle='-')
    plt.plot(history.history['val_loss'], label=f'LSTM Val {i+1}', linestyle='--')
plt.title('Learning Curves for LSTM Models')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig(f"{output_dir}/LSTM_Learning_Curves.png")
plt.show()

# Plot learning curves for NN models
plt.figure(figsize=(14, 8))
for i, history in enumerate(nn_history):
    plt.plot(history.history['loss'], label=f'NN Train {i+1}', linestyle='-')
    plt.plot(history.history['val_loss'], label=f'NN Val {i+1}', linestyle='--')
plt.title('Learning Curves for Neural Network Models')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig(f"{output_dir}/NN_Learning_Curves.png")
plt.show()


Calculation of model weights based on inverse RMSE for each forecasted month

In [19]:
import os
import pandas as pd

# Directory containing the monthly performance metric files
directory = 'Aggregated Approach Outputs'  
files = sorted([f for f in os.listdir(directory) if f.startswith('performance_metrics_overall_')])

# Initialise a list to store the results
monthly_weights = []

# Iterate over each file
for file in files:
    date_str = file.replace('performance_metrics_overall_', '').replace('.csv', '')
    
    # Load the performance metrics file
    file_path = os.path.join(directory, file)
    metrics_df = pd.read_csv(file_path)

    # Extract the RMSE values for each model
    rmse_xgboost = metrics_df['RMSE XGBoost'].values[0]
    rmse_lstm = metrics_df['RMSE LSTM'].values[0]
    rmse_nn = metrics_df['RMSE NN'].values[0]

    # Calculate the inverse RMSE and then normalize to get the weights
    inv_rmse_xgboost = 1 / rmse_xgboost
    inv_rmse_lstm = 1 / rmse_lstm
    inv_rmse_nn = 1 / rmse_nn

    # Sum of inverse RMSE values
    sum_inv_rmse = inv_rmse_xgboost + inv_rmse_lstm + inv_rmse_nn

    # Calculate the weights
    weight_xgboost = inv_rmse_xgboost / sum_inv_rmse
    weight_lstm = inv_rmse_lstm / sum_inv_rmse
    weight_nn = inv_rmse_nn / sum_inv_rmse

    # Append the results to the list
    monthly_weights.append({
        'Date': date_str,
        'XGBoost Weight': weight_xgboost,
        'LSTM Weight': weight_lstm,
        'NN Weight': weight_nn
    })

# Convert the results to a DataFrame for easy viewing and saving
weights_df = pd.DataFrame(monthly_weights)

# Display the weights per month
print(weights_df)

# Save the weights to a CSV file
weights_df.to_csv('Aggregated Approach Outputs/model_weights_per_month.csv', index=False)


         Date  XGBoost Weight  LSTM Weight  NN Weight
0  2024-01-01        0.362220     0.371043   0.266737
1  2024-02-01        0.417606     0.303097   0.279296
2  2024-03-01        0.588145     0.168127   0.243728


Analysis of correlation between model predictions

In [ ]:
import numpy as np
import pandas as pd

# The predictions from the XGBoost, LSTM, and Neural Network (NN) models are averaged across all bootstrapped predictions.
# This averaging is performed along the axis 0, which corresponds to the first axis (i.e., rows).
xgb_predictions = np.mean(xgboost_bootstrap_preds, axis=0)
lstm_predictions = np.mean(lstm_bootstrap_preds, axis=0)
nn_predictions = np.mean(nn_bootstrap_preds, axis=0)

# The averaged predictions from each model are combined into a single DataFrame.
# Each column in the DataFrame corresponds to the predictions from one model: XGBoost, LSTM, and NN.
predictions_df = pd.DataFrame({
    'XGBoost': xgb_predictions,
    'LSTM': lstm_predictions,
    'NN': nn_predictions
})

# A correlation matrix is calculated to understand the relationships between the predictions of different models.
# The correlation matrix quantifies how similar the predictions from different models are to each other.
correlation_matrix = predictions_df.corr()

# The calculated correlation matrix is printed to the console.
# This provides an immediate view of the degree to which the models' predictions are correlated.
print("Correlation Matrix:")
print(correlation_matrix)


Calculate performance metrics across demographic groups per country of citizenship, and across countries for overall asylum applications submitted

In [ ]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error

# Define the output directory where the results are saved
output_dir = "Aggregated Approach Outputs"

# Define the prediction dates used in the main code
prediction_dates = ['2024-01-01', '2024-02-01', '2024-03-01']  

# Function to calculate performance metrics
def calculate_metrics(y_true, y_pred):
    """
    Calculate the Mean Squared Error (MSE), Root Mean Squared Error (RMSE),
    Mean Absolute Error (MAE), and Median Absolute Error (MDAE).
    """
    mse = mean_squared_error(y_true, y_pred)
    rmse = mse ** 0.5
    mae = mean_absolute_error(y_true, y_pred)
    mdae = median_absolute_error(y_true, y_pred)
    return mse, rmse, mae, mdae

# Iterate through each prediction date to load and process the corresponding result files
for date in prediction_dates:
    # Load the results file for the current prediction date
    results_df = pd.read_csv(f'{output_dir}/country_level_aggregation_{date}.csv')

    # Extract true values and predictions
    y_true = results_df['True Values'].values
    y_pred_ensemble = results_df['Ensemble Prediction'].values
    y_pred_xgb = results_df['XGBoost Prediction'].values
    y_pred_lstm = results_df['LSTM Prediction'].values
    y_pred_nn = results_df['NN Prediction'].values

    # Calculate overall performance metrics across all countries for the current month
    overall_mse_ensemble, overall_rmse_ensemble, overall_mae_ensemble, overall_mdae_ensemble = calculate_metrics(y_true, y_pred_ensemble)
    overall_mse_xgb, overall_rmse_xgb, overall_mae_xgb, overall_mdae_xgb = calculate_metrics(y_true, y_pred_xgb)
    overall_mse_lstm, overall_rmse_lstm, overall_mae_lstm, overall_mdae_lstm = calculate_metrics(y_true, y_pred_lstm)
    overall_mse_nn, overall_rmse_nn, overall_mae_nn, overall_mdae_nn = calculate_metrics(y_true, y_pred_nn)

    # Store overall metrics across all countries in a DataFrame for the current month
    overall_metrics_df = pd.DataFrame({
        'Model': ['Ensemble', 'XGBoost', 'LSTM', 'NN'],
        'MSE': [overall_mse_ensemble, overall_mse_xgb, overall_mse_lstm, overall_mse_nn],
        'RMSE': [overall_rmse_ensemble, overall_rmse_xgb, overall_rmse_lstm, overall_rmse_nn],
        'MAE': [overall_mae_ensemble, overall_mae_xgb, overall_mae_lstm, overall_mae_nn],
        'MDAE': [overall_mdae_ensemble, overall_mdae_xgb, overall_mdae_lstm, overall_mdae_nn]
    }).round(2)

    # Save the overall performance metrics across all countries to a CSV file for the current month
    overall_metrics_df.to_csv(f'{output_dir}/overall_performance_metrics_{date}.csv', index=False)

    # Display the results for the current month
    print(f"Overall Performance Metrics Across All Countries for {date}:")
    print(overall_metrics_df)


Ensemble prediction errors across countries of citizenship

In [ ]:
import os
import pandas as pd

# Define the output directory where the results are saved
output_dir = "Aggregated Approach Outputs"

# Define the prediction dates used in the main code
prediction_dates = ['2024-01-01', '2024-02-01', '2024-03-01']  

# Initialise a DataFrame to hold overall errors across all countries
overall_errors = pd.DataFrame()

# Process each prediction date
for date in prediction_dates:
    # Load the aggregation results file for the current prediction date
    filepath = f'{output_dir}/country_level_aggregation_{date}.csv'
    if os.path.exists(filepath):
        data_df = pd.read_csv(filepath)

        # Calculate the error between True Values and Ensemble Prediction
        data_df['Ensemble Error'] = data_df['True Values'] - data_df['Ensemble Prediction']

        # Save the updated DataFrame with the Error column back to CSV
        data_df.to_csv(f'{output_dir}/country_level_aggregation_with_error_{date}.csv', index=False)
        
        # Append to the overall errors DataFrame
        overall_errors = pd.concat([overall_errors, data_df], ignore_index=True)

# Summarise errors by country and date
country_errors = overall_errors.groupby(['country', 'year_month'])['Ensemble Error'].sum().reset_index()

# Sort the DataFrame by 'year_month' and then by 'country'
country_errors = country_errors.sort_values(by=['year_month', 'country'])

# Save the sorted errors by country and date to a CSV file
country_errors.to_csv(f'{output_dir}/errors_by_country_and_date.csv', index=False)

# Calculate overall error across all countries and dates
total_error = overall_errors['Ensemble Error'].sum()
print(f"Total Error across all countries and dates: {total_error}")

# Save this overall error to a CSV for documentation
overall_error_df = pd.DataFrame({'Total Error': [total_error]})
overall_error_df.to_csv(f'{output_dir}/total_error_across_all_countries.csv', index=False)

# Display the DataFrame
print("Errors by country and date:")
print(country_errors)


Visualising asylum application forecasts with uncertainty intervals

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# Set the font to Times New Roman globally for all plots
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 12  

# Load the aggregated data from the final thesis dataset
data_agg = pd.read_csv('final_thesis_data.csv')

# Create an output directory for saving graphs
output_dir = "GHVT6_Outputs"
os.makedirs(output_dir, exist_ok=True)

# Ensure the 'year_month' column is in datetime format
data_agg['year_month'] = pd.to_datetime(data_agg['year_month'])

# Aggregate data at the country level by summing asylum applications for each country per month
country_data_agg = data_agg.groupby(['country', 'year_month']).agg({'asy_applications': 'sum'}).reset_index()

# Define the forecast dates and the corresponding forecast files
forecast_files = {
    '2024-01-01': 'Aggregated Approach Outputs/country_level_aggregation_2024-01-01.csv',
    '2024-02-01': 'Aggregated Approach Outputs/country_level_aggregation_2024-02-01.csv',
    '2024-03-01': 'Aggregated Approach Outputs/country_level_aggregation_2024-03-01.csv'
}
all_results_df = country_data_agg.copy()

# Loop through each forecast file, merging forecast data with the aggregated country data
for date, forecast_file in forecast_files.items():
    if os.path.exists(forecast_file):
        forecast_df = pd.read_csv(forecast_file)
        forecast_df['year_month'] = pd.to_datetime(date)  # Assign the forecast date
        forecast_df.rename(columns={
            'Ensemble Prediction': f'Ensemble Prediction_{date}',
            'Ensemble Lower CI': f'Lower CI_{date}',
            'Ensemble Upper CI': f'Upper CI_{date}'
        }, inplace=True)
        all_results_df = pd.merge(
            all_results_df,
            forecast_df[['country', 'year_month', f'Ensemble Prediction_{date}', f'Lower CI_{date}', f'Upper CI_{date}']],
            on=['country', 'year_month'],
            how='left'
        )
    else:
        print(f"Forecast file for {date} not found.")

# Save the merged data to a CSV file for further inspection
merged_data_filepath = os.path.join(output_dir, 'uncertainty_segmented_approach_country_level.csv')
all_results_df.to_csv(merged_data_filepath, index=False)
print(f"Merged data saved to {merged_data_filepath}")

# Create a directory for saving individual plots
individual_plots_dir = os.path.join(output_dir, "Forecasts' Plots with Uncertainty - Aggregated Forecasting Approach")
os.makedirs(individual_plots_dir, exist_ok=True)

# Loop through each country to generate and save forecast plots
for country, country_data in all_results_df.groupby('country'):
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Plot the true values of asylum applications over time
    ax.plot(country_data['year_month'], country_data['asy_applications'], label='True Values', linestyle='-', color='blue')
    
    # Plot the forecasted values and confidence intervals if forecast data is available
    forecast_mask = country_data['year_month'].isin(pd.to_datetime(list(forecast_files.keys())))
    if forecast_mask.any():
        ax.plot(country_data['year_month'], 
                country_data[['Ensemble Prediction_2024-01-01', 'Ensemble Prediction_2024-02-01', 'Ensemble Prediction_2024-03-01']]
                .ffill(axis=1).bfill(axis=1).iloc[:, -1], 
                label='Predictions', linestyle='--', color='red', marker=None)
        ax.fill_between(country_data['year_month'], 
                        country_data[['Lower CI_2024-01-01', 'Lower CI_2024-02-01', 'Lower CI_2024-03-01']].min(axis=1), 
                        country_data[['Upper CI_2024-01-01', 'Upper CI_2024-02-01', 'Upper CI_2024-03-01']].max(axis=1), 
                        alpha=0.2, color='red', label='95% CI')
    
    ax.set_title(f'Forecast for {country}')
    ax.set_xlabel('Year Month')
    ax.set_ylabel('Asylum Applications')
    ax.legend()
    ax.grid(True)
    
    # Format the x-axis for date presentation
    ax_inset = inset_axes(ax, width="30%", height="30%", loc='upper center', borderpad=2)
    ax_inset.plot(country_data['year_month'], country_data['asy_applications'], linestyle='-', color='blue')
    if forecast_mask.any():
        ax_inset.plot(country_data['year_month'], 
                      country_data[['Ensemble Prediction_2024-01-01', 'Ensemble Prediction_2024-02-01', 'Ensemble Prediction_2024-03-01']]
                      .ffill(axis=1).bfill(axis=1).iloc[:, -1], 
                      linestyle='--', color='red', marker=None)
        ax_inset.fill_between(country_data['year_month'], 
                              country_data[['Lower CI_2024-01-01', 'Lower CI_2024-02-01', 'Lower CI_2024-03-01']].min(axis=1), 
                              country_data[['Upper CI_2024-01-01', 'Upper CI_2024-02-01', 'Upper CI_2024-03-01']].max(axis=1), 
                              alpha=0.5, color='red')
    
    # Zoom in on the forecast period in the inset
    ax_inset.set_xlim(pd.to_datetime('2023-12-01'), pd.to_datetime('2024-03-31'))
    ax_inset.xaxis.set_major_locator(mdates.MonthLocator())
    ax_inset.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    fig.autofmt_xdate()

    # Save the plot for each country
    plt.tight_layout()
    plt.savefig(f'{individual_plots_dir}/Forecast_Comparison_{country}.png')
    plt.close()

print(f"Plots saved in directory: {individual_plots_dir}")
